In [1]:
#Zachary Elsisi Time Series Analysis

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from prophet import Prophet

In [3]:
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')

In [4]:
df = df.set_index(pd.DatetimeIndex(df['Created Date']))
del df['Created Date']

In [5]:
df

,Unique Key,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,City,Resolution Description,Borough,Open Data Channel Type
Created Date,,,,,,,,,,,
2011-04-06 00:00:00,20184537,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10002.0,NEW YORK,More than one complaint was received for this ...,MANHATTAN,UNKNOWN
2011-04-06 00:00:00,20184538,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,WINDOWS,RESIDENTIAL BUILDING,11236.0,BROOKLYN,The Department of Housing Preservation and Dev...,BROOKLYN,UNKNOWN
2011-04-06 00:00:00,20184539,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
2022-07-08 11:14:43,54732265,DSNY,Department of Sanitation,Dirty Condition,Trash,Sidewalk,10467.0,BRONX,The Department of Sanitation investigated this...,BRONX,PHONE
2011-04-06 00:00:00,20184540,HPD,Department of Housing Preservation and Develop...,NONCONST,VERMIN,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...
2011-04-06 00:00:00,20184532,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10468,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
2011-04-06 00:00:00,20184533,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10018,NEW YORK,More than one complaint was received for this ...,MANHATTAN,UNKNOWN
2011-04-06 00:00:00,20184534,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,STAIRS,RESIDENTIAL BUILDING,10460,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN


In [6]:
df.columns

Index(['Unique Key', 'Agency', 'Agency Name', 'Complaint Type', 'Descriptor',
       'Location Type', 'Incident Zip', 'City', 'Resolution Description',
       'Borough', 'Open Data Channel Type'],
      dtype='object')

In [7]:
df1 = df.loc['2022']

In [8]:
daily_complaints = df1.resample('D')['Unique Key'].nunique()
average_daily_complaints = daily_complaints.mean()
print(f"The average number of daily complaints received is: {average_daily_complaints:.2f}")

The average number of daily complaints received is: 8684.32


In [9]:
daily_complaints = df['Unique Key'].resample('D').count()
max_date = daily_complaints.idxmax()
print(f"On {max_date:%Y-%m-%d}, the maximum number of calls were received.")

On 2020-08-04, the maximum number of calls were received.


In [10]:
complaints_on_max_date = df[df.index.date == max_date.date()]
most_important_complaint_type = complaints_on_max_date['Complaint Type'].value_counts().idxmax()
print(f"On {max_date:%Y-%m-%d}, the most important complaint type was '{most_important_complaint_type}'.")

On 2020-08-04, the most important complaint type was 'Damaged Tree'.


In [11]:
monthly_calls = df.groupby(df.index.to_period('M'))['Unique Key'].nunique()
quietest_month = monthly_calls.idxmin().to_timestamp()
print(f"The quietest month historically is: {quietest_month.strftime('%B')}")

The quietest month historically is: August


In [16]:
result = sm.tsa.seasonal_decompose(daily_complaints, model='additive')
date_to_check = '2020-12-25'
rounded_seasonal_component = round(result.seasonal[date_to_check])
print(f"The rounded value of the seasonal component on {date_to_check} is: {rounded_seasonal_component}")

The rounded value of the seasonal component on 2020-12-25 is: 183


In [17]:
lagged_series = daily_complaints.shift(1)

# Calculate the autocorrelation
autocorrelation = daily_complaints.corr(lagged_series)

print(f"The autocorrelation with the number of calls the day prior is: {autocorrelation:.2f}")

The autocorrelation with the number of calls the day prior is: 0.75


In [93]:
train_set = daily_complaints.iloc[:-90]
test_set = daily_complaints.iloc[-90:]

In [94]:
train_set.head()

Created Date
2010-01-01    2942
2010-01-02    3958
2010-01-03    5676
2010-01-04    9763
2010-01-05    8735
Freq: D, Name: Unique Key, dtype: int64

In [95]:
train_set_prophet = daily_complaints.reset_index()
train_set_prophet.columns = ['ds', 'y']

In [96]:
train_set_prophet.head()

,ds,y
0,2010-01-01,2942
1,2010-01-02,3958
2,2010-01-03,5676
3,2010-01-04,9763
4,2010-01-05,8735


In [97]:
model = Prophet()
model.fit(train_set_prophet)

20:34:02 - cmdstanpy - INFO - Chain [1] start processing
20:34:03 - cmdstanpy - INFO - Chain [1] done processing


In [98]:
future = model.make_future_dataframe(periods=90,freq = 'd')
future.tail()

,ds
5049,2023-10-29
5050,2023-10-30
5051,2023-10-31
5052,2023-11-01
5053,2023-11-02


In [99]:
future.shape

(5054, 1)

In [100]:
forecast = model.predict(future)

In [101]:
forecast.columns

Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
       'weekly', 'weekly_lower', 'weekly_upper', 'yearly', 'yearly_lower',
       'yearly_upper', 'multiplicative_terms', 'multiplicative_terms_lower',
       'multiplicative_terms_upper', 'yhat'],
      dtype='object')

In [102]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2010-01-01,5931.551005,4184.098807,6984.079574,5931.551005,5931.551005,-349.171542,-349.171542,-349.171542,168.899548,168.899548,168.899548,-518.071089,-518.071089,-518.071089,0.0,0.0,0.0,5582.379463
1,2010-01-02,5930.596050,2919.220116,5770.203135,5930.596050,5930.596050,-1570.219604,-1570.219604,-1570.219604,-1095.034775,-1095.034775,-1095.034775,-475.184829,-475.184829,-475.184829,0.0,0.0,0.0,4360.376446
2,2010-01-03,5929.641096,2760.367433,5667.691459,5929.641096,5929.641096,-1742.994827,-1742.994827,-1742.994827,-1316.462036,-1316.462036,-1316.462036,-426.532791,-426.532791,-426.532791,0.0,0.0,0.0,4186.646269
3,2010-01-04,5928.686141,4807.022273,7660.626870,5928.686141,5928.686141,263.820580,263.820580,263.820580,636.716346,636.716346,636.716346,-372.895767,-372.895767,-372.895767,0.0,0.0,0.0,6192.506721
4,2010-01-05,5927.731187,4851.169775,7784.600000,5927.731187,5927.731187,381.136229,381.136229,381.136229,696.294051,696.294051,696.294051,-315.157822,-315.157822,-315.157822,0.0,0.0,0.0,6308.867416


In [103]:
preds = pd.DataFrame({'Prediction': forecast.yhat[-90:]})
preds.index = pd.to_datetime(forecast.ds[-90:])
preds.index.names = ['Date']
preds

,Prediction
Date,
2023-08-05,7780.661422
2023-08-06,7536.484583
2023-08-07,9465.004750
2023-08-08,9498.286358
2023-08-09,9304.462716
...,...
2023-10-29,7747.740649
2023-10-30,9704.760190
2023-10-31,9761.019139


In [104]:
test_set

Created Date
2023-05-07    9102
2023-05-08    9709
2023-05-09    9309
2023-05-10    9110
2023-05-11    9155
              ... 
2023-07-31    9921
2023-08-01    9813
2023-08-02    9245
2023-08-03    9128
2023-08-04     384
Freq: D, Name: Unique Key, Length: 90, dtype: int64

In [106]:
y_test = test_set.values
y_pred = preds['Prediction']
pd.DataFrame({'y_test': y_test, 'y_pred' : y_pred, 'diff':y_test - y_pred})

,y_test,y_pred,diff
Date,,,
2023-08-05,9102,7780.661422,1321.338578
2023-08-06,9709,7536.484583,2172.515417
2023-08-07,9309,9465.004750,-156.004750
2023-08-08,9110,9498.286358,-388.286358
2023-08-09,9155,9304.462716,-149.462716
...,...,...,...
2023-10-29,9921,7747.740649,2173.259351
2023-10-30,9813,9704.760190,108.239810
2023-10-31,9245,9761.019139,-516.019139


In [107]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print('MSE = ', mean_squared_error(y_test,y_pred))
print('RMSE = ', np.sqrt(mean_squared_error(y_test,y_pred)))
print('MAE = ', mean_absolute_error(y_test,y_pred))

MSE =  2173100.7311214292
RMSE =  1474.1440672883466
MAE =  937.4385423066217
